**IMPORTAÇÃO DAS BIBLIOTECAS**

In [6]:
# Importando as bibliotecas utilizadas e métodos ou atributos utilizados

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MinMaxScaler

**IMPORTAÇÃO DOS DADOS EM CSV**

In [7]:


from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# Atribuindo as planilhas a seu respectivo dataframe


df_demo = pd.read_excel('/content/drive/MyDrive/Inteli/Modulo_03/dataset/Cópia de BDIPMamaV11-INTELIDemograficosTt_DATA_LABELS_2023-01-24_1922.xlsx')
df_hist = pd.read_excel('/content/drive/MyDrive/Inteli/Modulo_03/dataset/BDIPMamaV11-INTELIHistopatologia_DATA_LABELS_2023-01-24_1924.xlsx')
df_peso = pd.read_excel('/content/drive/MyDrive/Inteli/Modulo_03/dataset/BDIPMamaV11-INTELIPesoEAltura_DATA_LABELS_2023-01-24_1926.xlsx')
df_tumor = pd.read_excel('/content/drive/MyDrive/Inteli/Modulo_03/dataset/BDIPMamaV11-INTELIRegistroDeTumo_DATA_LABELS_2023-01-24_1924.xlsx')


**SELEÇÃO PRIMÁRIA DE TABELAS**  

In [9]:
# Após debate inicial do grupo, decidiu-se seguir com as seguintes colunas em cada dataframe

df_demo = df_demo[['Record ID', 'Escolaridade', 'Idade do paciente ao primeiro diagnóstico', 'Sexo', 'Data da última informação sobre o paciente', 'Última informação do paciente', 'Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]', 'Já ficou grávida?', 'Idade na primeira gestação', 'Amamentou na primeira gestação?', 'Por quanto tempo amamentou?', 'Idade da primeira mentruação', 'Já fez uso de drogas?',	'Atividade Física',	'Consumo de tabaco',	'Consumo de álcool', 'Possui histórico familiar de câncer?', 'Regime de Tratamento']]


df_peso = df_peso[['Record ID','Peso', 'Altura (em centimetros)', 'IMC']]


df_tumor = df_tumor[['Record ID', 'Código da Topografia (CID-O)', 'Código da Morfologia de acordo com o CID-O', 'Estadio Clínico  ', 'Grupo de Estadio Clínico', 'Classificação TNM Clínico - T', 'Classificação TNM Clínico - N', 'Classificação TNM Clínico - M', 'Combinação dos Tratamentos Realizados no Hospital', 'Lateralidade do tumor', 'Tempo desde o diagnóstico até a primeira recidiva  ', 'Descrição da Morfologia de acordo com o CID-O  (CID-O - 3ª edição)', 'Descrição da Topografia ', 'Com recidiva à distância', 'Com recidiva regional', 'Com recidiva local']]


df_hist = df_hist.drop(columns='Indice H (Receptorde progesterona)')



***LIMPEZA DE LINHAS TOTALMENTE VAZIAS OU POBRES DE CONTEÚDO***

In [10]:
#não foi necessário fazer esse processo com o df_demo

#A lógica utilizada foi manter nos dataframes apenas as linhas que tenham pelo menos uma célula não nula, eliminando as linhas vazias

df_tumor = df_tumor[df_tumor.isnull().sum(axis=1) < 14]

df_peso = df_peso[df_peso.isnull().sum(axis=1) < 3]

df_hist = df_hist[df_hist.isnull().sum(axis=1) < 10]

print(df_tumor.shape)
print(df_peso.shape)
print(df_hist.shape)
print(df_demo.shape)
print(df_peso['Record ID'].value_counts())

(4677, 16)
(51334, 4)
(4787, 14)
(4272, 18)
12321.0    96
39366.0    83
30317.0    80
18944.0    75
16927.0    73
           ..
30261.0     1
72964.0     1
49325.0     1
50244.0     1
73132.0     1
Name: Record ID, Length: 3801, dtype: int64


*TRATAMENTO DA TABELA **PESO** PARA EVITAR REPETIÇÕES* // TRATAMENTO DE OUTLIERS

In [11]:
#SETANDO O NUMERO DE CASAS DECIMAIS
pd.set_option('display.precision',1)

menores = df_peso[(df_peso.IMC > 5)]
maiores = menores[(menores.IMC < 50)]


#Agrupando por ID e colocando a média 
df_peso = maiores[(maiores.IMC != np.inf)].groupby(['Record ID'], as_index=False).mean()

#pegar só a ultima ocorrencia

RETIRANDO LINHAS COM MAIS DE UMA OCORRÊNCIA, **MANTENDO O ÚLTIMO REGISTRO**

In [12]:
#Aqui utilizou-se o comando drop_duplicates com o parâmetro keep = last
#de forma que o dataframe retire todas as linhas que tenham o mesmo Record ID
#mantendo apenas a última linha na tabela, a fim de manter o agrupamento

df_hist = df_hist.drop_duplicates(subset=['Record ID'], keep='last')
df_tumor = df_tumor.drop_duplicates(subset=['Record ID'], keep='last')

print(df_peso['Record ID'].nunique())
print(df_hist['Record ID'].nunique())
print(df_tumor['Record ID'].nunique())
print(df_demo['Record ID'].nunique())

print('-----------------')

print(df_peso['Record ID'].value_counts().sum())
print(df_hist['Record ID'].value_counts().sum())
print(df_tumor['Record ID'].value_counts().sum())
print(df_demo['Record ID'].value_counts().sum())

3746
4272
4272
4272
-----------------
3746
4272
4272
4272


**MERGE DAS TABELAS**

In [13]:
#Fazendo o merge das tabelas com o comando merge()

df_merge = df_demo.merge(df_tumor)
df_merge = df_merge.merge(df_hist)
df_merge = df_merge.merge(df_peso)

df_dados = df_merge
df_dados.isnull().sum()

Record ID                                                                                           0
Escolaridade                                                                                      203
Idade do paciente ao primeiro diagnóstico                                                         169
Sexo                                                                                              139
Data da última informação sobre o paciente                                                          2
Última informação do paciente                                                                       2
Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]       2
Já ficou grávida?                                                                                2800
Idade na primeira gestação                                                                       2914
Amamentou na primeira gestação?                                                   



**SELEÇÃO DE COLUNAS ESPECÍFICAS PARA TRABALHAR**

In [14]:
#Após discussão da equipe, com base nos materiais que nos foram fornecidos para estudo do problema
#usando guidelines, entrevistas e artigos como referência
#decidimos utilizar as seguintes colunas no nosso modelo

dfz = df_dados[['Record ID', 'Idade do paciente ao primeiro diagnóstico', 'Última informação do paciente', 'Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]', 'Já ficou grávida?', 'Regime de Tratamento', 'Classificação TNM Clínico - M', 'Classificação TNM Clínico - N', 'Classificação TNM Clínico - T', 'Lateralidade do tumor', 'Com recidiva à distância', 'Com recidiva regional', 'Com recidiva local', 'Estadio Clínico  ', 'Combinação dos Tratamentos Realizados no Hospital']]
for coluna in dfz:
  print(f"{df_dados[coluna].unique(),coluna}---")



(array([  302.,   710.,   752., ..., 82131., 82205., 82240.]), 'Record ID')---
(array([51., 58., 56., 63., 42., 43., 52., 40., 60., 29., 36., 75., 45.,
       31., 30., 71., 48., 50., 39., 49., 61., 47., 55., 74., 37., 59.,
       32., 64., 44., 35., 57., 62., 54., 68., 33., 78., 27., 28., 46.,
       69., 80., 34., 82., 70., 41., 65., 73., 53., 72., 22., 66., 26.,
       38., 81., 79., 25., 24., 83., 86., 67., 84., 91., 76., 77., 89.,
       85., 87., 92., 23., 90., 95., 88., 98., nan, 96.]), 'Idade do paciente ao primeiro diagnóstico')---
(array(['Obito por câncer', 'Vivo, SOE', 'Vivo, com câncer',
       'Óbito por outras causas, SOE', nan], dtype=object), 'Última informação do paciente')---
(array([2225., 3294., 4153., ...,  391.,  366.,  138.]), 'Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]')---
(array([nan, 'Sim', 'Não'], dtype=object), 'Já ficou grávida?')---
(array([nan, 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Paliativo',
 

##**TRATAMENTO DE MISSINGS DAS COLUNAS SELECIONADAS**

In [15]:
#SETANDO O NUMERO DE CASAS DECIMAIS
pd.set_option('display.precision',0)

#Cria-se duas variações do dataframe utilizado
#Sendo elas respectivamente para fazer a codificação e normalização da coluna e retirar os outliers, afim de que a média calculada seja realista
tes = dfz.dropna(subset=['Última informação do paciente'])
tes2 = dfz.dropna(subset=['Última informação do paciente'])


tes2['Idade do paciente ao primeiro diagnóstico'].fillna(tes['Idade do paciente ao primeiro diagnóstico'].describe()['mean'], inplace= True)
#Aqui escolhemos substituir a idade com média temporariamente para poder fazer a analise dos dados e depois modificamos novamente para "0", para facilitar o reconhecimento de dados ausentes na hora da codificação
tes2['Já ficou grávida?'].fillna("Não Informado Gravida", inplace= True)
tes2['Regime de Tratamento'].fillna("Não Informado Tratamento", inplace= True)


tes['Idade do paciente ao primeiro diagnóstico'].fillna("0", inplace= True)
tes['Já ficou grávida?'].fillna("Não Informado Gravida", inplace= True)
tes['Regime de Tratamento'].fillna("Não Informado Tratamento", inplace= True)

#EMBORA TENHA SIDO UTILIZADO DF2 PARA ANALISES, A CONTINUIDADE DO CÓDIGO SE DARÁ COM O DF1 PORQUE ELE ESTÁ COM A IDADE ADAPTADA PARA SER CODIFICADA


/usr/local/lib/python3.9/dist-packages/pandas/core/generic.py:6392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)


**TRATAMENTO DE ALGUNS OUTLIERS DAS COLUNAS SELECIONADAS**

In [16]:
#ELIMINANDO PACIENTES ACIMA DE 90 ANOS PORQUE POSSUIMOS POUCAS AMOSTRAGENS ACIMA DESSA FAIXA ETÁRIA E PARA EVITAR UNDERFITTING
index_idade = tes2[ (tes2['Idade do paciente ao primeiro diagnóstico'] >= 90)].index
tes2.drop(index_idade, inplace = True)
tes.drop(index_idade, inplace = True)

#ELIMINANDO PACIENTES QUE POSSUEM REGISTRO DE TUMOR ABAIXO DE 20 DIAS PORQUE ISSO É MUITO POUCO E PROVAVELMENTE A DATA ESTÁ BUGADA
index_diastumor = tes2[ (tes2['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'] <= 20)].index
tes2.drop(index_diastumor, inplace = True)
tes.drop(index_diastumor, inplace = True)


/usr/local/lib/python3.9/dist-packages/pandas/core/frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


## **Estatística Descritiva**

**RECONHECIMENTO DE COLUNAS NUMÉRICAS X COLUNAS CATEGÓRICAS**

In [17]:
#Através do comando dtypes, identificamos quais são as tabelas categóricas e numéricas

cont = 0
listacolcat = []
for coluna in tes2.dtypes:
  if coluna == 'float64' or coluna == 'int64':
    tipo = "Coluna Númerica"

  elif coluna == object:
      tipo = "Coluna Categórica"
      listacolcat.append(tes2.columns[cont])
      
  print(f'{tes2.columns[cont]} é {tipo}\n========\n')
  cont+=1


Record ID é Coluna Númerica

Idade do paciente ao primeiro diagnóstico é Coluna Númerica

Última informação do paciente é Coluna Categórica

Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci] é Coluna Númerica

Já ficou grávida? é Coluna Categórica

Regime de Tratamento é Coluna Categórica

Classificação TNM Clínico - M é Coluna Categórica

Classificação TNM Clínico - N é Coluna Categórica

Classificação TNM Clínico - T é Coluna Categórica

Lateralidade do tumor é Coluna Categórica

Com recidiva à distância é Coluna Categórica

Com recidiva regional é Coluna Categórica

Com recidiva local é Coluna Categórica

Estadio Clínico   é Coluna Categórica

Combinação dos Tratamentos Realizados no Hospital é Coluna Categórica



  **ESTATÍSTICA DESCRITIVA DAS COLUNAS NÚMERICAS**
  

In [18]:
#Para as colunas numéricas, usamos o comando describe() para realizar a estatística descritiva


tes2.describe()


,Record ID,Idade do paciente ao primeiro diagnóstico,Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos [dt_pci]
count,3726,3726,3726
mean,49219,54,1501
std,20989,13,842
min,302,22,25
25%,30608,45,1008
50%,54701,54,1301
75%,67576,63,1837
max,82240,89,4503


**ESTATÍSTICA DESCRITIVA DAS COLUNAS CATEGÓRICAS**

In [19]:
#Para as colunas categóricas, usamos o comando value_counts() para realizar a estatística descritiva

for coluna in listacolcat:
  print(f'Coluna:{coluna}\n\n{tes2[coluna].value_counts()}\n\n\n') 

Coluna:Última informação do paciente

Vivo, SOE                       2536
Obito por câncer                 910
Vivo, com câncer                 218
Óbito por outras causas, SOE      62
Name: Última informação do paciente, dtype: int64



Coluna:Já ficou grávida?

Não Informado Gravida    2787
Sim                       928
Não                        11
Name: Já ficou grávida?, dtype: int64



Coluna:Regime de Tratamento

Terapia Adjuvante           1275
Não Informado Tratamento    1195
Terapia Neoadjuvante        1176
Paliativo                     55
Não fez quimioterapia         25
Name: Regime de Tratamento, dtype: int64



Coluna:Classificação TNM Clínico - M

0.0                                3249
1.0                                 445
Y: Na                                16
X - nao foi possivel determinar       7
1B                                    6
1A                                    3
Name: Classificação TNM Clínico - M, dtype: int64



Coluna:Classificação TNM Clínico - 

**3 GRÁFICOS RELACIONAIS ENTRE VARIÁVEIS ESCOLHIDAS PELO GRUPO**


Aqui relacionamos a idade do paciente e o tempo desde o último tumor

In [20]:
#Gráfico gerado através do lineplot
sns.lineplot(data=tes2, x='Idade do paciente ao primeiro diagnóstico', y='Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]')



<AxesSubplot:xlabel='Idade do paciente ao primeiro diagnóstico', ylabel='Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'>

Nesse outro gráfico relacionamos a situação do paciente com sua idade e explicitamos o regime de tratamento utilizado de cada um

In [21]:
#Gráfico gerado através do barplot

sns.barplot(data=tes2, x="Última informação do paciente", y="Idade do paciente ao primeiro diagnóstico", hue="Regime de Tratamento")

<AxesSubplot:xlabel='Última informação do paciente', ylabel='Idade do paciente ao primeiro diagnóstico'>

Por último, nesse gráfico tentamos enxergar a relação entre a lateralidade do tumor e o tempo desde o último tumor, e novamente tentamos explicitar qual foi o regime de tratamento utilizado

In [22]:
#Gráfico gerado através do catplot


sns.catplot(data=tes2, x="Lateralidade do tumor", y="Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]", hue='Regime de Tratamento', aspect=10.7/7.3, height=10)

# **CODIFICAÇÃO**

CODIFICAÇÃO IDADE (label) E ÚLTIMA INFORMAÇÃO (one-hot)

In [23]:
# #SETANDO O NUMERO DE CASAS DECIMAIS

# pd.set_option('display.precision',1)

# #transformando a coluna a seguir em inteiro
# tes['Idade do paciente ao primeiro diagnóstico'] = tes['Idade do paciente ao primeiro diagnóstico'].astype('int64')

# #dividindo as idades em intervalos, de forma que cada pessoa esteja em grupos com base nos intervalos estipulados
# tes['Idade COD'] = pd.cut(tes['Idade do paciente ao primeiro diagnóstico'], bins=[-1,1,30,45,60,89], labels=['-1','1','2','3','4'])
# print(tes.shape)
# #-1 -> NÃO INFORMADO
# # 1 = Idade entre 1~30 anos
# # 2 = Idade entre 30~45 anos
# # 3 = Idade entre 45~60 anos
# # 4 = Idade 60+ anos

#Setando casa decimais em 4
pd.set_option('display.precision',4)
#Transformando coluna em numerico
tes['Idade do paciente ao primeiro diagnóstico'] = tes['Idade do paciente ao primeiro diagnóstico'].astype('int64')
#Colocando mediana em valores vazios 
tes['Idade do paciente ao primeiro diagnóstico'] = tes['Idade do paciente ao primeiro diagnóstico'].replace(0, tes['Idade do paciente ao primeiro diagnóstico'].median())
#Scaler 
scaler_norm = MinMaxScaler()
tes['Idade Normalizada'] = scaler_norm.fit_transform((tes['Idade do paciente ao primeiro diagnóstico']).values.reshape(-1, 1))

tes[['Idade Normalizada','Idade do paciente ao primeiro diagnóstico']]
tes.drop(columns=['Idade do paciente ao primeiro diagnóstico'],inplace=True)

<ipython-input-23-3b0fcb96f3aa>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tes['Idade do paciente ao primeiro diagnóstico'] = tes['Idade do paciente ao primeiro diagnóstico'].astype('int64')
<ipython-input-23-3b0fcb96f3aa>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tes['Idade do paciente ao primeiro diagnóstico'] = tes['Idade do paciente ao primeiro diagnóstico'].replace(0, tes['Idade do paciente ao primeiro diagnóstico'].median())
<ipython-input-23-3b0fcb96f3aa>:25: SettingWithCopyWarning: 
A

In [24]:
#Aplicação de lógica de OneHotEncoder para a seguinte coluna

ohe = OneHotEncoder()
opa = ohe.fit_transform(tes['Última informação do paciente'].values.reshape(-1,1)).toarray()

dfohe = pd.DataFrame(opa, columns=['Óbito por Câncer','Vivo SOE', 'Vivo com Câncer', 'Óbito por outras causas SOE'])
dfohe.shape
df = pd.concat([tes,dfohe], axis=1)
df

,Record ID,Última informação do paciente,Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos [dt_pci],Já ficou grávida?,Regime de Tratamento,Classificação TNM Clínico - M,Classificação TNM Clínico - N,Classificação TNM Clínico - T,Lateralidade do tumor,Com recidiva à distância,Com recidiva regional,Com recidiva local,Estadio Clínico,Combinação dos Tratamentos Realizados no Hospital,Idade Normalizada,Óbito por Câncer,Vivo SOE,Vivo com Câncer,Óbito por outras causas SOE
0,302.0,Obito por câncer,2225.0,Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,Não,IIA,Cirurgia + Radio + Quimio + Hormonio,0.4328,1.0,0.0,0.0,0.0
1,710.0,"Vivo, SOE",3294.0,Não Informado Gravida,Terapia Adjuvante,0.0,1.0,3.0,Esquerda,Não,Sim,Sim,IIIA,Cirurgia + Quimioterapia,0.5373,0.0,1.0,0.0,0.0
2,752.0,"Vivo, SOE",4153.0,Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,Não,IIA,Outras combinações,0.5075,0.0,1.0,0.0,0.0
3,1367.0,Obito por câncer,1331.0,Não Informado Gravida,Não Informado Tratamento,0.0,1.0,1.0,Esquerda,Não,Sim,Não,IIA,Outras combinações,0.6119,1.0,0.0,0.0,0.0
4,1589.0,"Vivo, SOE",3290.0,Não Informado Gravida,Não Informado Tratamento,0.0,1.0,2.0,Direita,Não,Não,Não,IIB,Cirurgia + Radio + Quimio,0.2985,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,82123.0,"Vivo, SOE",380.0,Não Informado Gravida,Terapia Neoadjuvante,0.0,0.0,2.0,Esquerda,Não,Não,Não,IIA,Outras combinações,0.4627,NaN,NaN,NaN,NaN
3742,82124.0,Obito por câncer,138.0,Sim,Não Informado Tratamento,0.0,0.0,CDIS,Direita,Não,Não,Não,0.0,Quimioterapia,0.2836,NaN,NaN,NaN,NaN
3743,82131.0,Obito por câncer,900.0,Sim,Não Informado Tratamento,0.0,1.0,3.0,Direita,Não,Não,Não,IIIA,Cirurgia + Radioterapia,0.5522,NaN,NaN,NaN,NaN
3744,82205.0,Obito por câncer,538.0,Não Informado Gravida,Não Informado Tratamento,1.0,1.0,4D,Esquerda,Não,Não,Sim,IV,Cirurgia + Radio + Quimio,0.1045,NaN,NaN,NaN,NaN


CODIFICAÇÃO TEMPO DE SEGUIMENTO (LABEL)

In [25]:
# # Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita
# df = df.drop(df[df['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'] == 0].index)

# df['Tempo ultimo tumor'] = pd.cut(df['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'], bins=[0,365,730,1825,2920,4000], labels=['1','2','3','4','5'])


# df

# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

pd.set_option('display.precision',4)

df = df.drop(df[df['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]'] == 0].index)

scaler_norm_tempo = MinMaxScaler()
df['Tempo ultimo tumor'] = scaler_norm_tempo.fit_transform((df['Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]']).values.reshape(-1, 1))

df[['Tempo ultimo tumor']]
df.drop(columns='Tempo de seguimento (em dias) - desde o último tumor no caso de tumores múltiplos    [dt_pci]', inplace=True)
df

,Record ID,Última informação do paciente,Já ficou grávida?,Regime de Tratamento,Classificação TNM Clínico - M,Classificação TNM Clínico - N,Classificação TNM Clínico - T,Lateralidade do tumor,Com recidiva à distância,Com recidiva regional,Com recidiva local,Estadio Clínico,Combinação dos Tratamentos Realizados no Hospital,Idade Normalizada,Óbito por Câncer,Vivo SOE,Vivo com Câncer,Óbito por outras causas SOE,Tempo ultimo tumor
0,302.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,Não,IIA,Cirurgia + Radio + Quimio + Hormonio,0.4328,1.0,0.0,0.0,0.0,0.4913
1,710.0,"Vivo, SOE",Não Informado Gravida,Terapia Adjuvante,0.0,1.0,3.0,Esquerda,Não,Sim,Sim,IIIA,Cirurgia + Quimioterapia,0.5373,0.0,1.0,0.0,0.0,0.7300
2,752.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,Não,IIA,Outras combinações,0.5075,0.0,1.0,0.0,0.0,0.9218
3,1367.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,0.0,1.0,1.0,Esquerda,Não,Sim,Não,IIA,Outras combinações,0.6119,1.0,0.0,0.0,0.0,0.2916
4,1589.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,0.0,1.0,2.0,Direita,Não,Não,Não,IIB,Cirurgia + Radio + Quimio,0.2985,0.0,1.0,0.0,0.0,0.7291
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,82123.0,"Vivo, SOE",Não Informado Gravida,Terapia Neoadjuvante,0.0,0.0,2.0,Esquerda,Não,Não,Não,IIA,Outras combinações,0.4627,NaN,NaN,NaN,NaN,0.0793
3742,82124.0,Obito por câncer,Sim,Não Informado Tratamento,0.0,0.0,CDIS,Direita,Não,Não,Não,0.0,Quimioterapia,0.2836,NaN,NaN,NaN,NaN,0.0252
3743,82131.0,Obito por câncer,Sim,Não Informado Tratamento,0.0,1.0,3.0,Direita,Não,Não,Não,IIIA,Cirurgia + Radioterapia,0.5522,NaN,NaN,NaN,NaN,0.1954
3744,82205.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,1.0,1.0,4D,Esquerda,Não,Não,Sim,IV,Cirurgia + Radio + Quimio,0.1045,NaN,NaN,NaN,NaN,0.1146


CODIFICAÇÃO "JÁ FICOU GRÁVIDA" (ONE-HOT)

In [26]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Já ficou grávida?'].fillna("Não Informado Gravida", inplace= True)
ohegravida = ohe.fit_transform(df['Já ficou grávida?'].values.reshape(-1,1)).toarray()

df_ohegravida = pd.DataFrame(ohegravida, columns=['Não Grávida','Não informado Grávida','Grávida'])

df = pd.concat([df,df_ohegravida], axis=1)


CODIFICAÇÃO "REGIME DE TRATAMENTO" (ONE-HOT)

In [27]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Regime de Tratamento'].fillna("Não Informado Tratamento", inplace= True)
ohetrat = ohe.fit_transform(df['Regime de Tratamento'].values.reshape(-1,1)).toarray()


df_ohetrat = pd.DataFrame(ohetrat, columns=['Tratamento Não Informado','Não fez Quimioterapia','Paliativo','Terapia Adjuvante','Terapia Neoadjuvante'])
df = pd.concat([df,df_ohetrat], axis=1)


CODIFICAÇÃO "LATERALIDADE DO TUMOR"

In [28]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Lateralidade do tumor'].fillna("Não Informado Lateralidade", inplace= True)
ohelateral = ohe.fit_transform(df['Lateralidade do tumor'].values.reshape(-1,1)).toarray()


df_ohelat = pd.DataFrame(ohelateral, columns=['Bilateral','Direita','Esquerda','Lateralidade Não informada','Lateralidade não se aplica'])

df = pd.concat([df,df_ohelat], axis=1)
df


,Record ID,Última informação do paciente,Já ficou grávida?,Regime de Tratamento,Classificação TNM Clínico - M,Classificação TNM Clínico - N,Classificação TNM Clínico - T,Lateralidade do tumor,Com recidiva à distância,Com recidiva regional,...,Tratamento Não Informado,Não fez Quimioterapia,Paliativo,Terapia Adjuvante,Terapia Neoadjuvante,Bilateral,Direita,Esquerda,Lateralidade Não informada,Lateralidade não se aplica
0,302.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,710.0,"Vivo, SOE",Não Informado Gravida,Terapia Adjuvante,0.0,1.0,3.0,Esquerda,Não,Sim,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,752.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1367.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,0.0,1.0,1.0,Esquerda,Não,Sim,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1589.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,0.0,1.0,2.0,Direita,Não,Não,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,82123.0,"Vivo, SOE",Não Informado Gravida,Terapia Neoadjuvante,0.0,0.0,2.0,Esquerda,Não,Não,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3742,82124.0,Obito por câncer,Sim,Não Informado Tratamento,0.0,0.0,CDIS,Direita,Não,Não,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3743,82131.0,Obito por câncer,Sim,Não Informado Tratamento,0.0,1.0,3.0,Direita,Não,Não,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3744,82205.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,1.0,1.0,4D,Esquerda,Não,Não,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


CODIFICAÇÃO DAS RECIDIVAS (BINARY)

In [29]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Com recidiva local'].fillna("Não", inplace= True)
df['Recidiva Local'] = df['Com recidiva local']
df["Recidiva Local"] = np.where(df["Com recidiva local"] == "Não", 0, 1)

df['Com recidiva regional'].fillna("Não", inplace= True)
df['Recidiva Regional'] = df['Com recidiva regional']
df["Recidiva Regional"] = np.where(df["Com recidiva regional"] == "Não", 0, 1)

df['Com recidiva à distância'].fillna("Não", inplace= True)
df['Recidiva à distância'] = df['Com recidiva à distância']
df["Recidiva à distância"] = np.where(df["Com recidiva à distância"] == "Não", 0, 1)

df


,Record ID,Última informação do paciente,Já ficou grávida?,Regime de Tratamento,Classificação TNM Clínico - M,Classificação TNM Clínico - N,Classificação TNM Clínico - T,Lateralidade do tumor,Com recidiva à distância,Com recidiva regional,...,Terapia Adjuvante,Terapia Neoadjuvante,Bilateral,Direita,Esquerda,Lateralidade Não informada,Lateralidade não se aplica,Recidiva Local,Recidiva Regional,Recidiva à distância
0,302.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
1,710.0,"Vivo, SOE",Não Informado Gravida,Terapia Adjuvante,0.0,1.0,3.0,Esquerda,Não,Sim,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1,1,0
2,752.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,0.0,0.0,2.0,Esquerda,Não,Não,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,0,0
3,1367.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,0.0,1.0,1.0,Esquerda,Não,Sim,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0,1,0
4,1589.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,0.0,1.0,2.0,Direita,Não,Não,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3741,82123.0,"Vivo, SOE",Não Informado Gravida,Terapia Neoadjuvante,0.0,0.0,2.0,Esquerda,Não,Não,...,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0,0,0
3742,82124.0,Obito por câncer,Sim,Não Informado Tratamento,0.0,0.0,CDIS,Direita,Não,Não,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0
3743,82131.0,Obito por câncer,Sim,Não Informado Tratamento,0.0,1.0,3.0,Direita,Não,Não,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0,0,0
3744,82205.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,1.0,1.0,4D,Esquerda,Não,Não,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1,0,0


CODIFICAÇÃO "ESTADIO CLINICO" (LABEL)

In [30]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

 #label_encoder = LabelEncoder()
df['Estadio Clínico  '].fillna("IV", inplace= True)
df['Estadio Clínico  '].replace({0:"0"}, inplace= True)


# lista_i = df.index

# for i in lista_i:
#   df.loc[i, 'Estadio Clínico  '] = str(df.loc[i, 'Estadio Clínico  '])

# df['Estadio Clínico COD'] = label_encoder.fit_transform(df['Estadio Clínico  '])
# df['Estadio Clínico  '].unique()
# df['Estadio Clínico COD'].value_counts()
# 0 - 0
# I - 1
# IA - 2
# IB - 3
# IB1 - 4
# IB2- 5
# II - 6
# IIA - 7
# IIB - 8
# IIC- 9
# III- 10
# IIIA - 11
# IIIB - 12
# IIIC - 13
# IV - 14
# IVA - 15
# IVB - 16
# IVC - 17
# Y: NA - 18
# X : não foi possivel determinar - 19

#Mudado-----------------------------------------------------------------------------------

ohe_v1 =ohe.fit_transform(df['Estadio Clínico  '].values.reshape(-1,1)).toarray()
XoneCod = pd.DataFrame(ohe_v1, columns = ['IIA', 'IIIA', 'IIB', 'IV', 'IIIB', 'IIIC', 'I', 'IVB', 'IA', 'IB',
       'Y: NA', '0', 'II', 'X: não foi possível determinar', 'III', 'IVA',
       'IB1', 'IVC', 'IIC', 'IB2'])
df=pd.concat([df,XoneCod], axis=1)
df.drop(columns=['Estadio Clínico  '], inplace=True)


In [31]:
df.dropna(how='any',axis=0, inplace=True)

CODIFICAÇÃO "CLASSIFICAÇÃO TMN CLÍNICO" (ONE-HOT)

In [32]:
df['Classificação TNM Clínico - M'].replace({0:"0", 1:"1", 2:"2", 3:"3", 4:"4"}, inplace= True)

X3 =ohe.fit_transform(df['Classificação TNM Clínico - M'].values.reshape(-1,1)).toarray()
XoneCo = pd.DataFrame(X3, columns = ['Class_TMN_M_0', 'Class_TMN_M1', 'Class_TMN_MY','Class_TMN_MX','Class_TMN_M1B','Class_TMN_M_1A'])
df=pd.concat([df,XoneCo], axis=1)
df.drop(columns=['Classificação TNM Clínico - M'], inplace=True)
df

df['Classificação TNM Clínico - T'].replace({0:"0", 1:"1", 2:"2", 3:"3", 4:"4"}, inplace= True)

X4 =ohe.fit_transform(df['Classificação TNM Clínico - T'].values.reshape(-1,1)).toarray()
XoneC = pd.DataFrame(X4, columns = ['Class_TMN_T_1', 'Class_TMN_T2', 'Class_TMN_T3', 'Class_TMN_T4','Class_TMN_T1C', 'Class_TMN_TX', 'Class_TMN_T4B', 'Class_TMN_T3A', 'Class_TMN_T_1A',
 'Class_TMN_T_1B', 'Class_TMN_TY', 'Class_TMN_T4D', 'Class_TMN_TIS', 'Class_TMN_T4A', 'Class_TMN_T0', 'Class_TMN_T4C', 'Class_TMN_TCDIS', 'Class_TMN_T3B', 'Class_TMN_T2B', 'Class_TMN_T1MIC', 'Class_TMN_T_2A','Class_TMN_T_2B','Class_TMN_TCLIS'])
df=pd.concat([df,XoneC], axis=1)
df.drop(columns=['Classificação TNM Clínico - T'], inplace=True)
df.dropna(how='any',axis=0, inplace=True)
df

df['Classificação TNM Clínico - N'].replace({0:"0", 1:"1", 2:"2", 3:"3", 4:"4"}, inplace= True)

X5 =ohe.fit_transform(df['Classificação TNM Clínico - N'].values.reshape(-1,1)).toarray()
Xone = pd.DataFrame(X5, columns = ['Class_TMN_N_0', 'Class_TMN_N1', 'Class_TMN_N2', 'Class_TMN_N3', 'Class_TMN_NX', 'Class_TMN_N3C', 'Class_TMN_N2A', 'Class_TMN_N3A', 'Class_TMN_N_Y',
 'Class_TMN_N_1B', 'Class_TMN_N2B', 'Class_TMN_N3B', 'Class_TMN_N_1A'])
df=pd.concat([df,Xone], axis=1)
df.drop(columns=['Classificação TNM Clínico - N'], inplace=True)
df.dropna(how='any',axis=0, inplace=True)
df

,Record ID,Última informação do paciente,Já ficou grávida?,Regime de Tratamento,Lateralidade do tumor,Com recidiva à distância,Com recidiva regional,Com recidiva local,Combinação dos Tratamentos Realizados no Hospital,Idade Normalizada,...,Class_TMN_N3,Class_TMN_NX,Class_TMN_N3C,Class_TMN_N2A,Class_TMN_N3A,Class_TMN_N_Y,Class_TMN_N_1B,Class_TMN_N2B,Class_TMN_N3B,Class_TMN_N_1A
0,302.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,Esquerda,Não,Não,Não,Cirurgia + Radio + Quimio + Hormonio,0.4328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,710.0,"Vivo, SOE",Não Informado Gravida,Terapia Adjuvante,Esquerda,Não,Sim,Sim,Cirurgia + Quimioterapia,0.5373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,752.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,Esquerda,Não,Não,Não,Outras combinações,0.5075,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1367.0,Obito por câncer,Não Informado Gravida,Não Informado Tratamento,Esquerda,Não,Sim,Não,Outras combinações,0.6119,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1589.0,"Vivo, SOE",Não Informado Gravida,Não Informado Tratamento,Direita,Não,Não,Não,Cirurgia + Radio + Quimio,0.2985,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3681,81900.0,"Vivo, SOE",Sim,Terapia Adjuvante,Esquerda,Não,Não,Não,Cirurgia + Radio + Quimio + Hormonio,0.4627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3682,81901.0,"Vivo, SOE",Sim,Terapia Neoadjuvante,Esquerda,Não,Não,Não,Cirurgia + Radio + Quimio,0.4627,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3683,81907.0,"Vivo, SOE",Sim,Não Informado Tratamento,Direita,Não,Não,Não,Outras combinações,0.4627,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3684,81909.0,"Vivo, com câncer",Sim,Não Informado Tratamento,Esquerda,Não,Não,Não,Quimioterapia,0.4627,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


CODIFICAÇÃO "COMBINAÇÃO DE TRATAMENTOS" (ONE HOT)

In [33]:
# Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

df['Combinação dos Tratamentos Realizados no Hospital'].fillna("Nenhum tratamento", inplace= True)

#Aqui eu pego a coluna 'Combinação dos Tratamentos Realizados no Hospital' e faço um One Hot Encoding, porem há tratamentos que se repetem
#Nas mesmas linhas, então para resolver isso uso o split para separar as strings repetidas da celula 

splittest=df['Combinação dos Tratamentos Realizados no Hospital'] .str.split('+',expand=True)
t =splittest.rename(columns={0:'Cirurgia',1:'Radio',2:'Quimio',3:'Hormonio'})
t.insert(4,'Nenhum_Trantamento',np.nan)
t.insert(5,'Outras_combinacoes',np.nan)

#Aqui eu localizo a celula que está na coluna errada e mando ela para a coluna certa
#'Outras combinações'=='Outras combinações' 

t.loc[t['Cirurgia']=='Outras combinações','Outras_combinacoes'] = 'Outras_combinacoes'
t['Outras_combinacoes']=t['Outras_combinacoes'].fillna(0)

#Mesma logica aplicada acima.
t.loc[(t['Cirurgia']=='Quimioterapia')|(t['Cirurgia']=='Quimio'),'Quimio'] = 'Quimio'
t['Quimio']=t['Quimio'].fillna(0)

#Mesma logica aplicada acima.
t.loc[t['Cirurgia']=='Hormonio','Hormonio'] = 'Hormonio'

#Mesma logica aplicada acima.
t.loc[(t['Cirurgia']=='Radio')|(t['Cirurgia']=='Radioterapia'),'Radio'] = 'Radio'

#Mesma logica aplicada acima.
t.loc[(t['Cirurgia']=='Nenhum_Trantamento'),'Nenhum_Trantamento'] = 'Nenhum_Trantamento'

#Aqui eu substituo valores vazios por '0'
t['Nenhum_Trantamento']=t['Nenhum_Trantamento'].replace({np.nan:0})

t.loc[(t['Radio']==' Quimioterapia'),'Quimio'] = 'Quimio'

#Aqui eu transformo as variaveis em 0 e 1
t.loc[(t['Cirurgia']!='Cirurgia')&(t['Cirurgia']!=' Cirurgia')&(t['Cirurgia']!='Cirurgia ')&(t['Cirurgia']!=' Cirurgia '),'Cirurgia'] = 0
 
t['Outras_combinacoes']=t['Outras_combinacoes'].replace({'Outras_combinacoes':1})

t['Quimio']=t['Quimio'].replace({'Quimio':1,' Quimio ':1,' Quimio':1})

t['Hormonio']=t['Hormonio'].replace({None:0,' Hormonio':1})

t['Radio']=t['Radio'].replace({None:0,' Radio ':1,'Radio':1,'Radio ':1})

t.loc[(t['Radio']!=0)&(t['Radio']!=1),'Radio'] = 1

t['Radio']=t['Radio'].replace({' Quimioterapia':0,' Radioterapia':'Radio'})

t['Cirurgia']=t['Cirurgia'].replace({'Cirurgia':1,' Cirurgia':1,'Cirurgia ':1,' Cirurgia ':1})

t.loc[(t['Quimio']!=0)&(t['Quimio']!=1),'Quimio'] = 1

df=pd.concat([df,t], axis=1)
df.drop(columns=['Combinação dos Tratamentos Realizados no Hospital'], inplace=True)

df.head(10)

list(df)

['Record ID',
 'Última informação do paciente',
 'Já ficou grávida?',
 'Regime de Tratamento',
 'Lateralidade do tumor',
 'Com recidiva à distância',
 'Com recidiva regional',
 'Com recidiva local',
 'Idade Normalizada',
 'Óbito por Câncer',
 'Vivo SOE',
 'Vivo com Câncer',
 'Óbito por outras causas SOE',
 'Tempo ultimo tumor',
 'Não Grávida',
 'Não informado Grávida',
 'Grávida',
 'Tratamento Não Informado',
 'Não fez Quimioterapia',
 'Paliativo',
 'Terapia Adjuvante',
 'Terapia Neoadjuvante',
 'Bilateral',
 'Direita',
 'Esquerda',
 'Lateralidade Não informada',
 'Lateralidade não se aplica',
 'Recidiva Local',
 'Recidiva Regional',
 'Recidiva à distância',
 'IIA',
 'IIIA',
 'IIB',
 'IV',
 'IIIB',
 'IIIC',
 'I',
 'IVB',
 'IA',
 'IB',
 'Y: NA',
 '0',
 'II',
 'X: não foi possível determinar',
 'III',
 'IVA',
 'IB1',
 'IVC',
 'IIC',
 'IB2',
 'Class_TMN_M_0',
 'Class_TMN_M1',
 'Class_TMN_MY',
 'Class_TMN_MX',
 'Class_TMN_M1B',
 'Class_TMN_M_1A',
 'Class_TMN_T_1',
 'Class_TMN_T2',
 'Class_

CODIFICAÇÃO TNM - M (LABEL)

In [34]:
# # Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

# df['Classificação TNM Clínico - M'].fillna("X - nao foi possivel determinar", inplace= True)

# for i in lista_i:
#   df.loc[i, 'Classificação TNM Clínico - M'] = str(df.loc[i, 'Classificação TNM Clínico - M'])

# df['Classificação TNM - M'] = label_encoder.fit_transform(df['Classificação TNM Clínico - M'])

# df
# #0 - 0
# #1 - 1
# #2 - 1A
# #3 - 1B
# #4 - X - nao foi possivel determinar
# #5 - Y: na


# df['Classificação TNM - M'].value_counts()

# Mudado-------------------------------------------------------------------------------------------------------------------------------------------------------------

CODIFICAÇÃO TNM - N (LABEL)

In [35]:
# # Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita

# df['Classificação TNM Clínico - N'].fillna("X - nao foi possivel determinar", inplace= True)

# for i in lista_i:
#   df.loc[i, 'Classificação TNM Clínico - N'] = str(df.loc[i, 'Classificação TNM Clínico - N'])

# df['Classificação TNM - N'] = label_encoder.fit_transform(df['Classificação TNM Clínico - N'])


# # 0 - 0 
# # 1 - 1
# # 2 - 1A
# # 3 - 1B
# # 4 - 2
# # 5 - 2A
# # 6 - 2B
# # 7 - 3
# # 8 - 3A
# # 9 - 3B
# # 10 - 3C
# # 11 - X -nao foi possivel determinar
# # 12 - Y: Na


# df.isnull().sum()

# Mudado-------------------------------------------------------------------------------------------------------------------------------------------------------------

CODIFICAÇÃO TNM - T (LABEL)

In [36]:
# # Lógica semelhante a anterior é aplicada nessa coluna, garantindo a codificação bem feita


# df['Classificação TNM Clínico - T'].fillna("X - nao foi possivel determinar", inplace= True)

# for i in lista_i:
#   df.loc[i, 'Classificação TNM Clínico - T'] = str(df.loc[i, 'Classificação TNM Clínico - T'])

# df['Classificação TNM - T'] = label_encoder.fit_transform(df['Classificação TNM Clínico - T'])


# # 0 - 0
# # 1 - 1
# # 2 - 1A
# # 3 - 1B
# # 4 - 1C
# # 5 - 1MIC
# # 6 - 2
# # 7 - 2A
# # 8 - 2B
# # 9 - 3
# # 10 - 3A 
# # 11 - 3B
# # 12 - 4
# # 13 - 4A
# # 14 - 4B
# # 15 - 4C
# # 16 - 4D
# # 17 - CDIS
# # 18 - CLIS
# # 19 - IS
# # 20 - X - nao foi possivel determinar
# # 21 - Y: Na
 
# df['Classificação TNM - T'].value_counts()

# Mudado-------------------------------------------------------------------------------------------------------------------------------------------------------------

In [37]:
print(df.isnull().sum())
df.shape


Record ID                        0
Última informação do paciente    0
Já ficou grávida?                0
Regime de Tratamento             0
Lateralidade do tumor            0
                                ..
Radio                            0
Quimio                           0
Hormonio                         0
Nenhum_Trantamento               0
Outras_combinacoes               0
Length: 98, dtype: int64


(3666, 98)

In [38]:
# #Correção de bugs observados

# df.isnull().sum()




# df_x = df.dropna(subset=['Vivo com Câncer','Grávida', 'Tempo ultimo tumor'])

# df_x.shape


# # VER COM O JOSE-----------------------------------------------------------------------------------------



In [39]:
# display(df_x.head(5))

# **Criando Target**

In [40]:
print(df['Última informação do paciente'].isnull().sum())


0


In [41]:
#Retirando os missings dessa coluna
df_teste = df.drop(df['Última informação do paciente'].isnull().index)


In [42]:
print(df_teste['Última informação do paciente'].isnull().sum())


0


In [43]:
df_teste = df.assign(target = df['Record ID'])
print(df_teste['Última informação do paciente'].unique())
df_teste = df_teste.drop(df[df['Última informação do paciente'] == 'Óbito por outras causas, SOE'].index)
df_teste = df_teste.drop(df[df['Última informação do paciente'].isnull()].index)
print(df_teste['Última informação do paciente'].unique())



['Obito por câncer' 'Vivo, SOE' 'Vivo, com câncer'
 'Óbito por outras causas, SOE']
['Obito por câncer' 'Vivo, SOE' 'Vivo, com câncer']


In [44]:
lista_i = df_teste.index



In [45]:
def funcao_target(df, i):
  if(df.loc[i, 'Última informação do paciente'] == 'Vivo, com câncer'):
    if (df.loc[i, 'Recidiva Local'] == 1 or df.loc[i, 'Recidiva Regional'] == 1 or df.loc[i, 'Recidiva à distância'] == 1):
      return 0
    else:
      return 1
  elif(df.loc[i, 'Última informação do paciente'] == 'Vivo, SOE'):
    return 1

  elif(df.loc[i, 'Última informação do paciente'] == 'Obito por câncer'):
    return 0


In [46]:
for i in lista_i:
  df_teste.loc[i, 'target'] = funcao_target(df_teste, i)


In [47]:
print(df_teste['target'].isnull().sum())

0


In [48]:
df_teste['Tempo ultimo tumor']

0       0.4913
1       0.7300
2       0.9218
3       0.2916
4       0.7291
         ...  
3681    0.1168
3682    0.1052
3683    0.0831
3684    0.1114
3685    0.0784
Name: Tempo ultimo tumor, Length: 3604, dtype: float64

In [49]:
# df_teste.drop(columns=['Vivo SOE', 'Vivo com Câncer', 'Óbito por outras causas SOE', 'Óbito por Câncer'], inplace=True)
# df_teste.drop(columns=['Tempo ultimo tumor'], inplace=True)
# df_teste.drop(df_teste['Tempo ultimo tumor'].isnull().index, inplace=True)

# df_teste2 = df_teste['Tempo ultimo tumor'].isnull()
# b = df_teste.loc[df_teste['Tempo ultimo tumor'].isnull()].index
# #a = df_teste.loc[df_teste['Tempo ultimo tumor']]

# df_teste3 = df_teste.drop(df_teste.loc[df_teste['Tempo ultimo tumor'].isnull()].index)
# df_teste3['Tempo ultimo tumor'].isnull().sum()
df_teste.drop(df_teste.loc[df_teste['Tempo ultimo tumor'].isnull()].index, inplace=True)
df_teste.drop(df_teste.loc[df_teste['Vivo SOE'].isnull()].index, inplace=True)

In [50]:
# Código teste para verificar a existência de missings

lista_c = df_teste.columns

for i in lista_c:
  print(i)
  print(df_teste[i].isnull().sum())
  print("\n\n")


Record ID
0



Última informação do paciente
0



Já ficou grávida?
0



Regime de Tratamento
0



Lateralidade do tumor
0



Com recidiva à distância
0



Com recidiva regional
0



Com recidiva local
0



Idade Normalizada
0



Óbito por Câncer
0



Vivo SOE
0



Vivo com Câncer
0



Óbito por outras causas SOE
0



Tempo ultimo tumor
0



Não Grávida
0



Não informado Grávida
0



Grávida
0



Tratamento Não Informado
0



Não fez Quimioterapia
0



Paliativo
0



Terapia Adjuvante
0



Terapia Neoadjuvante
0



Bilateral
0



Direita
0



Esquerda
0



Lateralidade Não informada
0



Lateralidade não se aplica
0



Recidiva Local
0



Recidiva Regional
0



Recidiva à distância
0



IIA
0



IIIA
0



IIB
0



IV
0



IIIB
0



IIIC
0



I
0



IVB
0



IA
0



IB
0



Y: NA
0



0
0



II
0



X: não foi possível determinar
0



III
0



IVA
0



IB1
0



IVC
0



IIC
0



IB2
0



Class_TMN_M_0
0



Class_TMN_M1
0



Class_TMN_MY
0



Class_TMN_MX
0



Class_TMN_M1B
0



Class_T

# **Separando teste e treino**

In [51]:
#Através do comando dtypes, identificamos quais são as tabelas categóricas e numéricas

cont = 0
cont1 = 0
listacolcat = []
listacolcat1 = []
for coluna in df_teste.dtypes:
  if coluna == 'float64' or coluna == 'int64':
    tipo = "Coluna Númerica"
    listacolcat1.append(df_teste.columns[cont1])

  elif coluna == object:
      tipo = "Coluna Categórica"
      listacolcat.append(df_teste.columns[cont])
      
  # print(f'{df_teste.columns[cont]} é {tipo}\n========\n')
  cont+=1
  cont1+=1
print(f'{listacolcat1} é numerica')
print(f'{listacolcat} é categorica')

['Record ID', 'Idade Normalizada', 'Óbito por Câncer', 'Vivo SOE', 'Vivo com Câncer', 'Óbito por outras causas SOE', 'Tempo ultimo tumor', 'Não Grávida', 'Não informado Grávida', 'Grávida', 'Tratamento Não Informado', 'Não fez Quimioterapia', 'Paliativo', 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica', 'Recidiva Local', 'Recidiva Regional', 'Recidiva à distância', 'IIA', 'IIIA', 'IIB', 'IV', 'IIIB', 'IIIC', 'I', 'IVB', 'IA', 'IB', 'Y: NA', '0', 'II', 'X: não foi possível determinar', 'III', 'IVA', 'IB1', 'IVC', 'IIC', 'IB2', 'Class_TMN_M_0', 'Class_TMN_M1', 'Class_TMN_MY', 'Class_TMN_MX', 'Class_TMN_M1B', 'Class_TMN_M_1A', 'Class_TMN_T_1', 'Class_TMN_T2', 'Class_TMN_T3', 'Class_TMN_T4', 'Class_TMN_T1C', 'Class_TMN_TX', 'Class_TMN_T4B', 'Class_TMN_T3A', 'Class_TMN_T_1A', 'Class_TMN_T_1B', 'Class_TMN_TY', 'Class_TMN_T4D', 'Class_TMN_TIS', 'Class_TMN_T4A', 'Class_TMN_T0', 'Class_TMN_T4C', 'Class_T

In [52]:
df_teste.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3604 entries, 0 to 3685
Data columns (total 99 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Record ID                       3604 non-null   float64
 1   Última informação do paciente   3604 non-null   object 
 2   Já ficou grávida?               3604 non-null   object 
 3   Regime de Tratamento            3604 non-null   object 
 4   Lateralidade do tumor           3604 non-null   object 
 5   Com recidiva à distância        3604 non-null   object 
 6   Com recidiva regional           3604 non-null   object 
 7   Com recidiva local              3604 non-null   object 
 8   Idade Normalizada               3604 non-null   float64
 9   Óbito por Câncer                3604 non-null   float64
 10  Vivo SOE                        3604 non-null   float64
 11  Vivo com Câncer                 3604 non-null   float64
 12  Óbito por outras causas SOE     36

In [53]:
new_df_teste = df_teste.loc[(df_teste['Terapia Adjuvante'] == 1) | (df_teste['Terapia Neoadjuvante'] == 1)]

new_df_teste2 = new_df_teste[[
 'Idade Normalizada',
 'Tempo ultimo tumor',
 'Não Grávida',
 'Não informado Grávida',
 'Grávida',
 'Bilateral',
 'Direita',
 'Esquerda',
 'Lateralidade Não informada',
 'Lateralidade não se aplica',
 'IIA',
 'IIIA',
 'IIB',
 'IV',
 'IIIB',
 'IIIC',
 'I',
 'IVB',
 'IA',
 'IB',
 'Y: NA',
 '0',
 'II',
 'X: não foi possível determinar',
 'III',
 'IVA',
 'IB1',
 'IVC',
 'IIC',
 'IB2',
 'Class_TMN_M_0',
 'Class_TMN_M1',
 'Class_TMN_MY',
 'Class_TMN_MX',
 'Class_TMN_M1B',
 'Class_TMN_M_1A',
 'Class_TMN_T_1',
 'Class_TMN_T2',
 'Class_TMN_T3',
 'Class_TMN_T4',
 'Class_TMN_T1C',
 'Class_TMN_TX',
 'Class_TMN_T4B',
 'Class_TMN_T3A',
 'Class_TMN_T_1A',
 'Class_TMN_T_1B',
 'Class_TMN_TY',
 'Class_TMN_T4D',
 'Class_TMN_TIS',
 'Class_TMN_T4A',
 'Class_TMN_T0',
 'Class_TMN_T4C',
 'Class_TMN_TCDIS',
 'Class_TMN_T3B',
 'Class_TMN_T2B',
 'Class_TMN_T1MIC',
 'Class_TMN_T_2A',
 'Class_TMN_T_2B',
 'Class_TMN_TCLIS',
 'Class_TMN_N_0',
 'Class_TMN_N1',
 'Class_TMN_N2',
 'Class_TMN_N3',
 'Class_TMN_NX',
 'Class_TMN_N3C',
 'Class_TMN_N2A',
 'Class_TMN_N3A',
 'Class_TMN_N_Y',
 'Class_TMN_N_1B',
 'Class_TMN_N2B',
 'Class_TMN_N3B',
 'Class_TMN_N_1A',
 'target'
]]



In [54]:
from sklearn.model_selection import train_test_split

df_adjuvante = new_df_teste2.loc[(df_teste['Terapia Adjuvante'] == 1)]
df_neoadjuvante = new_df_teste2.loc[(df_teste['Terapia Neoadjuvante'] == 1)]

# x=new_df_teste2
# y=new_df_teste['target']

x_adjuvante = df_adjuvante.drop(columns='target')
y_adjuvante = df_adjuvante['target']

x_neoadjuvante = df_neoadjuvante.drop(columns='target')
y_neoadjuvante = df_neoadjuvante['target']


#KNN
# x_knn = df_teste.drop(columns='Última informação do paciente')
# x_knn = df_teste[['Não Grávida', 'Grávida', 'Tratamento Não Informado', 'Não fez Quimioterapia', 'Paliativo', 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica','Estadio Clínico COD', 'C', 'C+Q', 'C+R+Q', 'C+R+Q+H', 'C+R', 'Nenhum Tratamento', 'Outras Combinações', 'Q', 'R', 'R+Q', 'Classificação TNM - M', 'Classificação TNM - N','Classificação TNM - T']]
# x_knn = df_teste[['Idade COD', 'Óbito por Câncer', 'Vivo SOE', 'Vivo com Câncer', 'Óbito por outras causas SOE', 'Tempo ultimo tumor', 'Não Grávida', 'Não informado Grávida', 'Grávida', 'Tratamento Não Informado', 'Não fez Quimioterapia', 'Paliativo', 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica', 'Recidiva Local', 'Recidiva Regional', 'Recidiva à distância', 'Estadio Clínico COD', 'C', 'C+Q', 'C+R+Q', 'C+R+Q+H', 'C+R', 'Nenhum Tratamento', 'Outras Combinações', 'Q', 'R', 'R+Q', 'Classificação TNM - M', 'Classificação TNM - N', 'Classificação TNM - T']]
# x_knn = x
# y_knn = df_teste['Última informação do paciente']
# y_knn = new_df_teste['target']



# x_treino_knn, x_teste_knn, y_treino_knn, y_teste_knn = train_test_split(x_knn, y_knn,
#                                                                         test_size=0.2,
#                                                                         random_state=1)

#ADJUVANTE

x_knn_adjuvante = x_adjuvante
y_knn_adjuvante = y_adjuvante

x_treino_knn_adjuvante, x_teste_knn_adjuvante, y_treino_knn_adjuvante, y_teste_knn_adjuvante = train_test_split(x_knn_adjuvante, y_knn_adjuvante,
                                                                                                                test_size=0.2,
                                                                                                                random_state=1)

#NEOADJUVANTE

x_knn_neoadjuvante = x_neoadjuvante
y_knn_neoadjuvante = y_neoadjuvante

x_treino_knn_neoadjuvante, x_teste_knn_neoadjuvante, y_treino_knn_neoadjuvante, y_teste_knn_neoadjuvante = train_test_split(x_knn_neoadjuvante, y_knn_neoadjuvante,
                                                                                                                test_size=0.2,
                                                                                                                random_state=1)


#Naive Bayes
# x_nb = df_teste.drop(columns='Última informação do paciente')
# x_nb = df_teste[['Não Grávida', 'Grávida', 'Tratamento Não Informado', 'Não fez Quimioterapia', 'Paliativo', 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica','Estadio Clínico COD', 'C', 'C+Q', 'C+R+Q', 'C+R+Q+H', 'C+R', 'Nenhum Tratamento', 'Outras Combinações', 'Q', 'R', 'R+Q', 'Classificação TNM - M', 'Classificação TNM - N','Classificação TNM - T']]
# x_nb = df_teste[['Idade COD', 'Óbito por Câncer', 'Vivo SOE', 'Vivo com Câncer', 'Óbito por outras causas SOE', 'Tempo ultimo tumor', 'Não Grávida', 'Não informado Grávida', 'Grávida', 'Tratamento Não Informado', 'Não fez Quimioterapia', 'Paliativo', 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica', 'Recidiva Local', 'Recidiva Regional', 'Recidiva à distância', 'Estadio Clínico COD', 'C', 'C+Q', 'C+R+Q', 'C+R+Q+H', 'C+R', 'Nenhum Tratamento', 'Outras Combinações', 'Q', 'R', 'R+Q', 'Classificação TNM - M', 'Classificação TNM - N', 'Classificação TNM - T']]
# x_nb = x

# y_nb = df_teste['Última informação do paciente']
# y_nb = new_df_teste['target']



# x_treino_nb, x_teste_nb, y_treino_nb, y_teste_nb = train_test_split(x_nb, y_nb,
#                                                                         test_size=0.2,
#                                                                         random_state=1)


#ADJUVANTE

x_nb_adjuvante = x_adjuvante
y_nb_adjuvante = y_adjuvante

x_treino_nb_adjuvante, x_teste_nb_adjuvante, y_treino_nb_adjuvante, y_teste_nb_adjuvante = train_test_split(x_nb_adjuvante, y_nb_adjuvante,
                                                                                                                test_size=0.2,
                                                                                                                random_state=1)

#NEOADJUVANTE

x_nb_neoadjuvante = x_neoadjuvante
y_nb_neoadjuvante = y_neoadjuvante

x_treino_nb_neoadjuvante, x_teste_nb_neoadjuvante, y_treino_nb_neoadjuvante, y_teste_nb_neoadjuvante = train_test_split(x_nb_neoadjuvante, y_nb_neoadjuvante,
                                                                                                                test_size=0.2,
                                                                                                                random_state=1)


#Random Forest
# x_rf = df_teste.drop(columns='Última informação do paciente')
# x_rf = df_teste[['Tratamento Não Informado', 'Não fez Quimioterapia', 'Paliativo', 'Terapia Adjuvante', 'Terapia Neoadjuvante', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica','Estadio Clínico COD', 'C', 'C+Q', 'C+R+Q', 'C+R+Q+H', 'C+R', 'Nenhum Tratamento', 'Outras Combinações', 'Q', 'R', 'R+Q', 'Classificação TNM - M', 'Classificação TNM - N','Classificação TNM - T']]
# x_rf = df_teste[['Idade COD', 'Tempo ultimo tumor', 'Não Grávida', 'Não informado Grávida', 'Grávida', 'Bilateral', 'Direita', 'Esquerda', 'Lateralidade Não informada', 'Lateralidade não se aplica', 'Estadio Clínico COD', 'Classificação TNM - M', 'Classificação TNM - N', 'Classificação TNM - T']]
# x_rf = x

# y_rf = df_teste['Última informação do paciente']
# y_rf = new_df_teste['target']


# x_treino_rf, x_teste_rf, y_treino_rf, y_teste_rf = train_test_split(x_rf, y_rf,
#                                                                         test_size=0.2,
#                                                                         random_state=1)



#ADJUVANTE

x_rf_adjuvante = x_adjuvante
y_rf_adjuvante = y_adjuvante

x_treino_rf_adjuvante, x_teste_rf_adjuvante, y_treino_rf_adjuvante, y_teste_rf_adjuvante = train_test_split(x_rf_adjuvante, y_rf_adjuvante,
                                                                                                                test_size=0.2,
                                                                                                                random_state=1)

#NEOADJUVANTE

x_rf_neoadjuvante = x_neoadjuvante
y_rf_neoadjuvante = y_neoadjuvante

x_treino_rf_neoadjuvante, x_teste_rf_neoadjuvante, y_treino_rf_neoadjuvante, y_teste_rf_neoadjuvante = train_test_split(x_rf_neoadjuvante, y_rf_neoadjuvante,
                                                                                                                test_size=0.2,
                                                                                                                random_state=1)


In [55]:
from sklearn.ensemble import RandomForestClassifier


In [56]:
modelo_adjuvante = RandomForestClassifier(n_estimators=10)
modelo_adjuvante.fit(x_adjuvante, y_adjuvante)

RandomForestClassifier(n_estimators=10)

In [57]:
modelo_neoadjuvante = RandomForestClassifier(n_estimators=10)
modelo_neoadjuvante.fit(x_neoadjuvante, y_neoadjuvante)

RandomForestClassifier(n_estimators=10)

In [58]:
#feature_importances_ para encontrar a relação de cada coluna com o modelo
#Sejam inteligentes em entender o pq de cada coluna ter cada relação
lista = new_df_teste2.columns
lista_i = modelo_adjuvante.feature_importances_

for i in range(0, len(lista)-1):
  print(lista[i])
  print(lista_i[i])
  print("\n\n")

Idade Normalizada
0.21417401662127608



Tempo ultimo tumor
0.27599922645748914



Não Grávida
0.006024948212845459



Não informado Grávida
0.01617676447815022



Grávida
0.012204510376622079



Bilateral
0.00220806001071088



Direita
0.02084035479642783



Esquerda
0.025301874523522283



Lateralidade Não informada
0.0



Lateralidade não se aplica
0.015317809369880434



IIA
0.0014830993652204297



IIIA
0.005291344874329004



IIB
0.01436766811127989



IV
0.001258363111056936



IIIB
0.0



IIIC
0.0



I
0.0



IVB
0.011712434319339634



IA
0.017930742126014667



IB
0.0



Y: NA
0.000857190183433719



0
0.011509110926624582



II
0.015691727979643785



X: não foi possível determinar
0.023080290396562942



III
0.06469098679481333



IVA
0.002735823403448362



IB1
0.0003792204775887441



IVC
0.0



IIC
0.0



IB2
0.00042098008928259195



Class_TMN_M_0
0.007838465372773345



Class_TMN_M1
0.009699730002193168



Class_TMN_MY
1.4196775946224483e-06



Class_TMN_MX
3.976722908

In [59]:
#feature_importances_ para encontrar a relação de cada coluna com o modelo
#Sejam inteligentes em entender o pq de cada coluna ter cada relação
lista = new_df_teste2.columns
lista_i = modelo_neoadjuvante.feature_importances_

for i in range(0, len(lista)-1):
  print(lista[i])
  print(lista_i[i])
  print("\n\n")

Idade Normalizada
0.21268675386296523



Tempo ultimo tumor
0.33193687817163414



Não Grávida
0.0008593999419804273



Não informado Grávida
0.01335390264238664



Grávida
0.01403808701773843



Bilateral
0.0003608044091051242



Direita
0.022203631223980363



Esquerda
0.0183964408364792



Lateralidade Não informada
0.0



Lateralidade não se aplica
0.006901694079677256



IIA
0.003322598023955057



IIIA
0.0008948659970359624



IIB
0.00435320007668249



IV
0.0006138392880416211



IIIB
0.0006495489491344963



IIIC
0.002043228032056585



I
0.0006383045004613116



IVB
0.019501422436319714



IA
0.024764935195712413



IB
0.0



Y: NA
0.0008763034084215045



0
0.016490789030401097



II
0.015414566907819399



X: não foi possível determinar
0.008098076786060667



III
0.07569734618139676



IVA
0.0003943845866176096



IB1
4.9551980347899824e-05



IVC
0.0



IIC
0.0009531303627371222



IB2
0.0013121929066790042



Class_TMN_M_0
0.01117714316856408



Class_TMN_M1
0.01243098066

#Método KNN

**Método KNN Adjuvante**

In [60]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report # metricas de validação

knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(x_treino_knn_adjuvante, y_treino_knn_adjuvante.squeeze())
y_pred_treino_knn_adjuvante = knn.predict(x_treino_knn_adjuvante)
y_pred_knn_adjuvante = knn.predict(x_teste_knn_adjuvante)
print('Acc treino: ', knn.score(x_treino_knn_adjuvante, y_treino_knn_adjuvante ))
print('Acc teste: ', knn.score(x_teste_knn_adjuvante, y_teste_knn_adjuvante.squeeze() ))
print(classification_report(y_teste_knn_adjuvante, y_pred_knn_adjuvante))

Acc treino:  0.87
Acc teste:  0.828
              precision    recall  f1-score   support

         0.0       0.53      0.20      0.30        44
         1.0       0.85      0.96      0.90       206

    accuracy                           0.83       250
   macro avg       0.69      0.58      0.60       250
weighted avg       0.79      0.83      0.80       250



**Método KNN Neoadjuvante**

In [61]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report # metricas de validação

knn = KNeighborsClassifier(n_neighbors=6)
knn.fit(x_treino_knn_neoadjuvante, y_treino_knn_neoadjuvante.squeeze())
y_pred_treino_knn_neoadjuvante = knn.predict(x_treino_knn_neoadjuvante)
y_pred_knn_neoadjuvante = knn.predict(x_teste_knn_neoadjuvante)
print('Acc treino: ', knn.score(x_treino_knn_neoadjuvante, y_treino_knn_neoadjuvante ))
print('Acc teste: ', knn.score(x_teste_knn_neoadjuvante, y_teste_knn_neoadjuvante.squeeze() ))
print(classification_report(y_teste_knn_neoadjuvante, y_pred_knn_neoadjuvante))

Acc treino:  0.7980241492864983
Acc teste:  0.7017543859649122
              precision    recall  f1-score   support

         0.0       0.57      0.51      0.54        78
         1.0       0.76      0.80      0.78       150

    accuracy                           0.70       228
   macro avg       0.67      0.66      0.66       228
weighted avg       0.70      0.70      0.70       228



#Método Random Forest

**Método Random Forest Adjuvante**

In [62]:
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth = 7)
rf.fit(x_treino_rf_adjuvante, y_treino_rf_adjuvante)


y_pred_treino_rf_adjuvante = rf.predict(x_treino_rf_adjuvante)
y_pred_rf_adjuvante = rf.predict(x_teste_rf_adjuvante)


print( rf.score(x_treino_rf_adjuvante, y_treino_rf_adjuvante) )
print( rf.score(x_teste_rf_adjuvante, y_teste_rf_adjuvante) )
print(classification_report(y_teste_knn_adjuvante, y_pred_knn_adjuvante))

0.882
0.84
              precision    recall  f1-score   support

         0.0       0.53      0.20      0.30        44
         1.0       0.85      0.96      0.90       206

    accuracy                           0.83       250
   macro avg       0.69      0.58      0.60       250
weighted avg       0.79      0.83      0.80       250



**Método Random Forest Neoadjuvante**

In [63]:
from sklearn.metrics import classification_report 
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(max_depth = 7)
rf.fit(x_treino_rf_neoadjuvante, y_treino_rf_neoadjuvante)


y_pred_treino_rf_neoadjuvante = rf.predict(x_treino_rf_neoadjuvante)
y_pred_rf_neoadjuvante = rf.predict(x_teste_rf_neoadjuvante)


print( rf.score(x_treino_rf_neoadjuvante, y_treino_rf_neoadjuvante) )
print( rf.score(x_teste_rf_neoadjuvante, y_teste_rf_neoadjuvante) )
print(classification_report(y_teste_knn_neoadjuvante, y_pred_knn_neoadjuvante))

0.8364434687156971
0.7763157894736842
              precision    recall  f1-score   support

         0.0       0.57      0.51      0.54        78
         1.0       0.76      0.80      0.78       150

    accuracy                           0.70       228
   macro avg       0.67      0.66      0.66       228
weighted avg       0.70      0.70      0.70       228



#Método Naive Bayes

In [64]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

**Método Naive Bayes Adjuvante**

In [65]:
metodo_nb = GaussianNB()
metodo_nb.fit(x_treino_nb_adjuvante, y_treino_nb_adjuvante)
y_pred_treino_nb_adjuvante = metodo_nb.predict(x_treino_nb_adjuvante)
y_pred_nb_adjuvante = metodo_nb.predict(x_teste_nb_adjuvante)
acc_treino_nb_adjuvante = accuracy_score(y_treino_nb_adjuvante, y_pred_treino_nb_adjuvante)
acc_teste_nb_adjuvante = accuracy_score(y_teste_nb_adjuvante, y_pred_nb_adjuvante) 
print('Acc de treino: ', acc_treino_nb_adjuvante)
print('Acc de teste: ', acc_teste_nb_adjuvante)

Acc de treino:  0.249
Acc de teste:  0.252


**Método Naive Bayes Neoadjuvante**

In [66]:
metodo_nb = GaussianNB()
metodo_nb.fit(x_treino_nb_neoadjuvante, y_treino_nb_neoadjuvante)
y_pred_treino_nb_neoadjuvante = metodo_nb.predict(x_treino_nb_neoadjuvante)
y_pred_nb_neoadjuvante = metodo_nb.predict(x_teste_nb_neoadjuvante)
acc_treino_nb_neoadjuvante = accuracy_score(y_treino_nb_neoadjuvante, y_pred_treino_nb_neoadjuvante)
acc_teste_nb_neoadjuvante = accuracy_score(y_teste_nb_neoadjuvante, y_pred_nb_neoadjuvante) 
print('Acc de treino: ', acc_treino_nb_neoadjuvante)
print('Acc de teste: ', acc_teste_nb_neoadjuvante)

Acc de treino:  0.3809001097694841
Acc de teste:  0.3991228070175439


O método Naive Bayes não respondeu bem às colunas escolhidas por nós, tendo acurácias abaixo dos 40%. Por isso, descartamos a utilização do Naive Bayes, por enquanto. Tanto o KNN quanto o Random Forest tiveram maior acurácia para o tratamento adjuvante, mas quando comparamos os dois, percebe-se que o Random Forest apresenta acuracidade ainda maior, sem caracterizar o overfitting. Por isso, escolheu-se o Random Forest como modelo candidato.
